In [22]:
# enrich data from trusted web sources.

import bs4
import json
import pandas
import pathlib
import pydash
import requests
import time

class enrich_data:

    def __init__(self, json_file):

        self.dictionary = dict()
        with open(json_file) as work:
            self.work = json.load(work)

    def pike_cooper_data(self):

        pike_cooper_id = pydash.get(self.work, 'pike-cooper')
        if pike_cooper_id:
            pike_cooper_csv = pathlib.Path.cwd().parents[1] / 'sources' / 'pike-cooper' / 'pike-cooper.csv'
            pike_cooper_data = pandas.read_csv(pike_cooper_csv)
            pike_cooper_dict = pike_cooper_data.to_dict('records')
            pike_cooper_dict = [x for x in pike_cooper_dict if x['pike-cooper identifier'] == int(pike_cooper_id)]

            # list of dicts needs to be inverted to dict of lists.

            self.dictionary['pike_cooper'] = pike_cooper_dict

    def oflc_data(self):

        oflc_id = pydash.get(self.work, 'oflc')

        if len(oflc_id):

            time.sleep(4)
            oflc_address = 'https://www.classification.gov.au/titles/'+oflc_id 

            oflc_data = requests.get(oflc_address).text
            oflc_data = oflc_data.split('Industry details')[1].replace('\n', '')
            oflc_data = oflc_data.split('<div class="views-row">')[1]
            oflc_data = oflc_data.split('</div></div></div>')[0]
            oflc_data = oflc_data.split('</div></div>')

            oflc_dict = dict()
            for o in oflc_data:
                o = o.split('<div class="field-content">')
                o = [bs4.BeautifulSoup(x, "lxml").text for x in o]
                oflc_dict[o[0]] = o[1]

            self.dictionary['oflc'] = oflc_dict

auth = [x for x in (pathlib.Path.cwd().parents[1] / 'authority').iterdir() if x.suffix == '.json']
for x in auth:
    work = enrich_data(x)
    work.pike_cooper_data()
    work.oflc_data()

    print(work.dictionary)


/home/temp/.local/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


{'pike_cooper': [{'label': 'Picnic at Hanging Rock (1975)', 'pike-cooper identifier': 436, 'year': 1975, 'title': 'Picnic at Hanging Rock', 'director': 'Peter Weir', 'duration': 115.0, 'work_uuid': 'cb97ec8c-7fb6-4c79-be7a-cd365c42f3d2', 'agent_uuid': 'c5d53fce-8b3e-4f2d-b79e-2f35c51a725d'}], 'oflc': {'Classification date': '1 August 1975', 'Year of production': 'Not Specified', 'Classification': 'G', 'Consumer advice': 'Not Specified', 'Category': 'Films', 'Category detail': 'Review - Public Exhibition', 'Media type': '35MM', 'Version': 'Not Specified', 'Duration': '118 minutes', 'Director/Creator': 'UNKNOWN', 'Producer': 'J & H MCELROY', 'Production company': 'Not Specified', 'Country of origin': 'AUSTRALIA', 'Language': 'ENGLISH', 'Applicant': 'MGM/BEF FILM DIST', 'File number': 'T84/2576', 'Classification number': '0030686F2140', 'Alternative titles': 'Not Specified'}}
{'oflc': {'Classification date': '14 December 1998', 'Year of production': 'Not Specified', 'Classification': 'R 1